## Json

Primero, quiero crear el archivo json donde guardaré la informacion sobre los intents, y las respuestas del chatbot

In [ ]:
import json

def guardar_json(datos, filename):
    '''creacion de funcion para guardar
    diccionario de conocimiento en formato json'''
    archivo=open(filename,"w")
    json.dump(datos,archivo,indent=4)

biblioteca = {
    "intents": [
        {
            "tag": "saludo",
            "patterns":  ["Hola", "Buenos días", "Buenas tardes", "Buenas noches", "Buenas"],
            "responses": ["¡Hola! Soy un asistente de predicción preventivo de Alzheimer, \nPuedo ayudarte con cualquier pregunta que tengas sobre el formulario de abajo"]
        },
        {
            "tag":"agradecimientos",
            "patterns":["gracias","muchas gracias","muy amable","se lo agradezco","muy agradecido","gracias por su tiempo"],
            "responses":["No hay de que, me alegro de haberle podido ser de ayuda"]
        },
        {
            "tag": "iniciar",
            "patterns":["Iniciemos", "Empecemos", "Comencemos","iniciar","comenzar", "empezar"],
            "responses":["De acuerdo, empecemos. Completa el formulario de abajo y presiona en 'Obtener diagnostico', no dejes ningun campo sin responder!"]
        }
    ]
}

# Guardado de diccionario de conocimiento en formato json.
guardar_json(biblioteca, 'intents.json')
biblioteca

{'intents': [{'tag': 'saludo',
   'patterns': ['Hola',
    'Buenos días',
    'Buenas tardes',
    'Buenas noches',
    'Buenas'],
   'responses': ['¡Hola! Soy un asistente de predicción preventivo de Alzheimer, \nPuedo ayudarte con cualquier pregunta que tengas sobre el formulario de abajo']},
  {'tag': 'agradecimientos',
   'patterns': ['gracias',
    'muchas gracias',
    'muy amable',
    'se lo agradezco',
    'muy agradecido',
    'gracias por su tiempo'],
   'responses': ['No hay de que, me alegro de haberle podido ser de ayuda']},
  {'tag': 'iniciar',
   'patterns': ['Iniciemos',
    'Empecemos',
    'Comencemos',
    'iniciar',
    'comenzar',
    'empezar'],
   'responses': ["De acuerdo, empecemos. Completa el formulario de abajo y presiona en 'Obtener diagnostico', no dejes ningun campo sin responder!"]}]}

## Modelo

### Modelo de chatbot

In [ ]:
import nltk
import json
import random
import numpy as np
import pickle
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# Cargar el archivo JSON
with open('intents.json') as file:
    intents = json.load(file)

# Inicializar el stemmer y el encoder
stemmer = PorterStemmer()
encoder = LabelEncoder() # Similar al Integer Encoding

# Recursos necesarios de nltk
nltk.download('punkt')
# Intentar descargar punkt_tab con manejo de errores
try:
    nltk.download('punkt_tab')
except Exception as e:
    print(f"Advertencia: No se pudo descargar 'punkt_tab': {e}")
    print("Continuando sin este recurso...")

# Preparar los datos
patterns = []  # Base de conocimientos por cada intención
tags = []      # Intents

# Tokenización y Stemming
def tokenize_and_stem(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
    return " ".join(words)

# Llenar las listas con los datos del JSON
for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(tokenize_and_stem(pattern))
        tags.append(intent['tag']) # Intent

# Crear el Bag of Words (BoW)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(patterns).toarray()

# Codificar las etiquetas
y_train = encoder.fit_transform(tags)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Crear la red neuronal
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(tags)), activation='softmax'))

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.3478 - loss: 1.1098
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4820 - loss: 1.0770
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9031 - loss: 1.0132
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9431 - loss: 0.9689 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.9261
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.8627 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.8420 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.7915
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.7515 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 0.6939 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.6556 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.000

### Alzheimer

In [ ]:
!pip install kagglehub[pandas-datasets] imbalanced-learn category_encoders

import joblib
import kagglehub
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from kagglehub import KaggleDatasetAdapter
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.1 MB/s eta 0:00:00


#### Cargado y tratamiento de datos

In [ ]:
# Set the path to the file you'd like to load
file_path = "alzheimers_prediction_dataset.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ankushpanday1/alzheimers-prediction-dataset-global",
  file_path,
)
df.head(3)

<ipython-input-6-962c07011c00>:5: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


100%|██████████| 9.65M/9.65M [00:00<00:00, 16.6MB/s]


,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
0,Spain,90,Male,1,33.0,Medium,Never,Occasionally,No,No,...,Healthy,High,Retired,Single,No,Low,Medium,High,Urban,No
1,Argentina,72,Male,7,29.9,Medium,Former,Never,No,No,...,Healthy,Medium,Unemployed,Widowed,No,High,Low,High,Urban,No
2,South Africa,86,Female,19,22.9,High,Current,Occasionally,No,Yes,...,Average,Medium,Employed,Single,No,Low,Medium,High,Rural,No


In [ ]:
df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})
df['Diabetes'] = df['Diabetes'].map({'No':0, 'Yes':1})
df['Hypertension'] = df['Hypertension'].map({'No':0, 'Yes':1})
df['Employment Status'] = df['Employment Status'].map({'Unemployed':0, 'Employed':1, 'Retired':2})
df['Marital Status'] = df['Marital Status'].map({'Single':0, 'Married':1, 'Widowed':2})
df['Urban vs Rural Living'] = df['Urban vs Rural Living'].map({'Urban':0, 'Rural':1})

# Variables categiricas Ordinales (Orden específico)
df['Cholesterol Level'] = df['Cholesterol Level'].map({'Normal': 0, 'High': 1})
df['Family History of Alzheimer’s'] = df['Family History of Alzheimer’s'].map({'No': 0, 'Yes': 1})
df['Genetic Risk Factor (APOE-ε4 allele)'] = df['Genetic Risk Factor (APOE-ε4 allele)'].map({'No': 0, 'Yes': 1})

df['Physical Activity Level'] = df['Physical Activity Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
df['Smoking Status'] = df['Smoking Status'].map({'Never': 0, 'Former': 1, 'Current': 2})
df['Alcohol Consumption'] = df['Alcohol Consumption'].map({'Never': 0, 'Occasionally': 1, 'Regularly': 2})
df['Depression Level'] = df['Depression Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
df['Sleep Quality'] = df['Sleep Quality'].map({'Poor': 0, 'Average': 1, 'Good': 2})
df['Dietary Habits'] = df['Dietary Habits'].map({'Unhealthy': 0, 'Average': 1, 'Healthy': 2})
df['Air Pollution Exposure'] = df['Air Pollution Exposure'].map({ 'Low':0 ,'Medium':1,'High':2})
df['Social Engagement Level'] = df['Social Engagement Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
df['Stress Levels'] = df['Stress Levels'].map({'Low': 0, 'Medium': 1, 'High': 2})
df['Income Level'] = df['Income Level'].map({'Low': 0, 'Medium': 1, 'High': 2})

# Aplicar Target Encoding a 'Country'
df['Alzheimer’s Diagnosis'] = df['Alzheimer’s Diagnosis'].map({'No': 0, 'Yes': 1}) #¡Target de la data!
target_enc = TargetEncoder()
df['Country'] = target_enc.fit_transform(df['Country'],df['Alzheimer’s Diagnosis'])

scaler = StandardScaler()
df_N = df.copy()

#### Entrenamiento y exportación del modelo

In [ ]:
# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_N.drop(['Country', 'Alzheimer’s Diagnosis'], axis=1))
y = df_N['Alzheimer’s Diagnosis']

# Manejo del desbalanceo de clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Modelos con hiperparámetros ajustados
model_alzheimer = RandomForestClassifier(n_estimators=1200, max_depth=40, min_samples_split=2, random_state=42)

model_alzheimer.fit(X_train, y_train)
joblib.dump(model, "alzheimer_model.pkl")
joblib.dump(scaler, "alzheimer_scaler.pkl")
score_alzheimer = model_alzheimer.score(X_test, y_test) * 100


In [ ]:
print(np.unique(y_train, return_counts=True))
model_alzheimer.classes_

(array([0, 1]), array([34880, 34832]))


array([0, 1])

## Chatbot

Creación del chatbot.

In [ ]:
!pip install gradio openai

import gradio as gr
import nltk
import pandas as pd
import pickle
import time
import joblib
from tensorflow.keras.models import load_model
from nltk.stem import PorterStemmer
from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


### Codigo

In [ ]:
# Cargar modelos
model = load_model('chatbot_model.h5')
modeloAlzheimer = joblib.load("alzheimer_model.pkl")
# Cargar vectorizador y encoder
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
# Cargar scaler
alzheimer_scaler = joblib.load("alzheimer_scaler.pkl")

API_KEY = "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"

client = OpenAI(
  base_url ="https://openrouter.ai/api/v1",
  api_key = API_KEY,
)

# Inicializar el stemmer
stemmer = PorterStemmer()

# Recursos necesarios de nltk
nltk.download('punkt')
# Intentar descargar punkt_tab con manejo de errores
try:
    nltk.download('punkt_tab')
except Exception as e:
    print(f"Advertencia: No se pudo descargar 'punkt_tab': {e}")
    print("Continuando sin este recurso...")

# Tokenización y Stemming
def tokenize_and_stem(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
    return " ".join(words)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#### model chatbot propio

In [ ]:
def chatbot_propio(msg, confidence_threshold=0.87):
    # Preprocesar el texto de entrada
    text_words = tokenize_and_stem(msg)
    bow = vectorizer.transform([text_words]).toarray()

    # Obtener probabilidades de cada clase
    probabilities = model.predict(bow, verbose=0)[0]  # verbose=0 para evitar mensajes de TF
    confidence = np.max(probabilities)
    predicted_class = np.argmax(probabilities)

    print(f"Texto: '{msg}'")
    print(f"Confianza: {confidence:.4f}, Umbral: {confidence_threshold}")

    if confidence >= confidence_threshold:
        tag = encoder.inverse_transform([predicted_class])[0]

        # Buscar una respuesta dentro de la categoría
        for intent in intents['intents']:
            if intent['tag'] == tag:
                return intent['responses'][0]
    else:
        return "confianza baja"  # Retorna cadena vacía para usar el modelo preentrenado

#### model alzheimer

In [ ]:
def alzheimer_model(in1, in2, in3, in4, in5, in6, in7, in8, in9, in10, in11, in12, in13, in14, in15, in16, in17, in18, in19, in20, in21, in22, in23):

   # Cargar modelo
  # Insertar codigo aqui//

  # Tratar los datos
  # Insertar codigo aqui//
  Col= ['Age', 'Gender', 'Education Level', 'BMI','Physical Activity Level', 'Smoking Status', 'Alcohol Consumption','Diabetes', 'Hypertension', 'Cholesterol Level',
       'Family History of Alzheimer’s', 'Cognitive Test Score','Depression Level', 'Sleep Quality', 'Dietary Habits','Air Pollution Exposure', 'Employment Status', 'Marital Status',
       'Genetic Risk Factor (APOE-ε4 allele)', 'Social Engagement Level','Income Level', 'Stress Levels', 'Urban vs Rural Living']
  dato = [in1, in2, in3, in4, in5, in6, in7, in8, in9, in10, in11, in12, in13, in14, in15, in16, in17, in18, in19, in20, in21, in22, in23]

  df = pd.DataFrame([dato],columns=Col)

  df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})
  df['Diabetes'] = df['Diabetes'].map({'No':0, 'Yes':1})
  df['Hypertension'] = df['Hypertension'].map({'No':0, 'Yes':1})
  df['Employment Status'] = df['Employment Status'].map({'Unemployed':0, 'Employed':1, 'Retired':2})
  df['Marital Status'] = df['Marital Status'].map({'Single':0, 'Married':1, 'Widowed':2})
  df['Urban vs Rural Living'] = df['Urban vs Rural Living'].map({'Urban':0, 'Rural':1})

  # Variables categiricas Ordinales (Orden específico)
  df['Cholesterol Level'] = df['Cholesterol Level'].map({'Normal': 0, 'High': 1})
  df['Family History of Alzheimer’s'] = df['Family History of Alzheimer’s'].map({'No': 0, 'Yes': 1})
  df['Genetic Risk Factor (APOE-ε4 allele)'] = df['Genetic Risk Factor (APOE-ε4 allele)'].map({'No': 0, 'Yes': 1})

  df['Physical Activity Level'] = df['Physical Activity Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
  df['Smoking Status'] = df['Smoking Status'].map({'Never': 0, 'Former': 1, 'Current': 2})
  df['Alcohol Consumption'] = df['Alcohol Consumption'].map({'Never': 0, 'Occasionally': 1, 'Regularly': 2})
  df['Depression Level'] = df['Depression Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
  df['Sleep Quality'] = df['Sleep Quality'].map({'Poor': 0, 'Average': 1, 'Good': 2})
  df['Dietary Habits'] = df['Dietary Habits'].map({'Unhealthy': 0, 'Average': 1, 'Healthy': 2})
  df['Air Pollution Exposure'] = df['Air Pollution Exposure'].map({ 'Low':0 ,'Medium':1,'High':2})
  df['Social Engagement Level'] = df['Social Engagement Level'].map({'Low': 0, 'Medium': 1, 'High': 2})
  df['Stress Levels'] = df['Stress Levels'].map({'Low': 0, 'Medium': 1, 'High': 2})
  df['Income Level'] = df['Income Level'].map({'Low': 0, 'Medium': 1, 'High': 2})

  df = alzheimer_scaler.transform(df)

  # Obtener predicción del modelo
  diagnostico = modeloAlzheimer.predict(df)
  categorias = ["Incierto", "Negativo", "Positivo"]

  indice_max = np.argmax(diagnostico)
  diagnostico = categorias[indice_max]


  res = f"Diagnostido para alzheimer: {diagnostico}\nPrecisión de diagnostico: 77%\n\nConsultar con su médico"
  return res



#### model chatbot deepseek

In [ ]:
def deepseek(history):
  chat = client.chat.completions.create(
      model="deepseek/deepseek-r1:free",
      messages=history
  )
  return chat.choices[0].message.content

### Gradio

In [ ]:
def chat(msg, history):
  # Volver a guardar el historial y actualizarlo con el nuevo mensaje
  keys_to_keep = ["role", "content"]
  history = [{k: d[k] for k in keys_to_keep if k in d} for d in history]
  history.append({"role": "user", "content": msg})

  # Implementación de lógica
  res = chatbot_propio(msg)
  if res == "confianza baja": res = deepseek(history)
  for i in range(len(res)):
    time.sleep(0.009)
    yield res[:i+1]

In [ ]:
with gr.Blocks() as demo:

  with gr.Row():

    with gr.Column():
      gr.Markdown("<center><h1>Chatbot</h1></center>")
      chatI = gr.ChatInterface(fn=chat, type="messages", title="predictor de alzheimer")
      gr.Markdown("<br><br><center><h2>Formulario de diagnostico</h2></center><br><br>")

      with gr.Group():
        in1 = gr.Number(label="Edad")
        in2 = gr.Radio(["Male", "Female"], show_label=True, label="Sexo")
        in3 = gr.Number(label="Nivel de educación", info="años estudiados")
        in4 = gr.Number(label="Indice de masa corporal")
        in5 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Actividad física")
        in6 = gr.Radio(["Current", "Never", "Former"], show_label=True, label="Es fumador?")
        in7 = gr.Radio(["Never", "Regularly", "Occasionally"], show_label=True, label="Cada cuanto toma alcohol?")
        in8 = gr.Radio(["Yes", "No"], show_label=True, label="Diabetes")
        in9 = gr.Radio(["Yes", "No"], show_label=True, label="Hypertension")
        in10 = gr.Radio(["Normal", "High"], show_label=True, label="Nivel de colesterol")
        in11 = gr.Radio(["Yes", "No"], show_label=True, label="Hay historial de alzheimer en la familia?")
        in12 = gr.Number(label="Puntaje de prueba cognitiva? (30-100)")
        in13 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Nivel de depresion")
        in14 = gr.Radio(["Poor", "Average", "Good"], show_label=True, label="Calidad de sueño")
        in15 = gr.Radio(["Healthy", "Average", "Unhealthy"], show_label=True, label="Habito de dieta")
        in16 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Exposicion a contaminacion del aire")
        in17 = gr.Radio(["Employed", "Unemployed", "Retired"], show_label=True, label="Estatus de empleo")
        in18 = gr.Radio(["Single", "Married", "Widowed"], show_label=True, label="Estatus marital")
        in19 = gr.Radio(["Yes", "No"], show_label=True, label="Factor de riesgo genetico (APOE ε4)", info="La Apolipoproteína E (APOE) es la variante ε4?")
        in20 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Nivel de interacción social", info="nivel de cuánto interactúa con los demás en su vida diaria")
        in21 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Nivel de ingresos")
        in22 = gr.Radio(["Low", "Medium", "High"], show_label=True, label="Nivel de estres")
        in23 = gr.Radio(["Urban", "Rural"], show_label=True, label="Vive en zona...")
      with gr.Row(equal_height=True):
        btn_diagnostico = gr.Button("Obtener Diagnostico", variant="primary")
        txt_diagnostico = gr.Textbox(placeholder="El resultado aparecerá aquí")
  listOfComponents = [in1, in2, in3, in4, in5, in6, in7, in8, in9, in10, in11, in12, in13, in14, in15, in16, in17, in18, in19, in20, in21, in22, in23]
  btn_diagnostico.click(alzheimer_model, listOfComponents, txt_diagnostico)


demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b07a163446ed24487c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Pruebas del modelo para un usuario sin alzheimer y con alzheimer

In [ ]:
print(alzheimer_model(21, "Male", 17, 20, "Low", "Never", "Occasionally", "No", "No", "Normal", "Yes", 90, "Low", "Average", "Average", "Low", "Unemployed", "Single", "No", "Medium", "Low", "Medium", "Urban"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Diagnostido para alzheimer: Negativo
Precisión de diagnostico: 77%

Consultar con su médico


In [ ]:
print(alzheimer_model(61, "Male", 15, 22.9, "High", "Former", "Regularly", "No", "No", "Normal", "No", 89, "Low", "Average", "Healthy", "High", "Employed", "Married", "No", "Medium", "High", "High", "Urban"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Diagnostido para alzheimer: Positivo
Precisión de diagnostico: 77%

Consultar con su médico
